In [2]:
import zipfile
import os
def unzip_file(zip_path, extract_to='.'):
    """
    Unzips the specified archive to the target directory.

    Parameters:
    zip_path (str): Path to the .zip file.
    extract_to (str): Directory where files will be extracted. Default is the current directory.
    """
    # Check if the specified path is a valid zip file
    if zipfile.is_zipfile(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f'Files extracted to {os.path.abspath(extract_to)}')
    else:
        print(f"{zip_path} is not a valid zip file.")

unzip_file('DATASET.zip')

Files extracted to /content


In [12]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf

print(tf.__version__)
test_labels = pd.read_csv(f'test_labels.csv')
train_labels = pd.read_csv(f'train_labels.csv')

data_gen = ImageDataGenerator(
    rescale=1.0/255,   # Normalize pixel values between 0 and 1
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    shear_range=0.15,
    zoom_range=0.15,
    fill_mode='nearest'
)

train_generator = data_gen.flow_from_directory(
    directory='DATASET/train',
    target_size=(100, 100),   # Resize images to 100x100
    color_mode='rgb',         # Specify that images are in RGB
    batch_size=32,            # Adjust this batch size as needed
    class_mode='categorical', # Use 'categorical' for multi-class classification
    subset='training'         # Use the 'training' subset
)

test_data_gen = ImageDataGenerator(rescale=1.0/255)  # Only rescaling for test data
test_generator = test_data_gen.flow_from_directory(
    directory='DATASET/test',
    target_size=(100, 100),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical'
)

emotion_labels = {
    0: "Surprise",
    1: "Fear",
    2: "Disgust",
    3: "Happiness",
    4: "Sadness",
    5: "Anger",
    6: "Neutral"
}


2.17.0
Found 12271 images belonging to 7 classes.
Found 3067 images belonging to 7 classes.


In [ ]:
# test train generator
import matplotlib.pyplot as plt

# Generate a batch of images and labels from the generator
images, labels = next(train_generator)

# Plot a few images to check if augmentation and loading are correct
plt.figure(figsize=(10, 10))
for i in range(9):  # Display the first 9 images in the batch
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])  # Display image
    plt.title(f"Class: {emotion_labels[np.argmax(labels[i])]}")
    plt.title(f"Class: {emotion_labels[np.argmax(labels[i])]}")
    plt.axis('off')  # Hide axis

plt.show()


In [15]:
# create the model and train
from tensorflow.keras.applications import EfficientNetV2B0  # Importing EfficientNetV2B0 for a smaller model version
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the EfficientNetV2 base model with imagenet weights
base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)  # Adding batch normalization
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)  # Adding batch normalization
x = Dropout(0.3)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)  # Adding batch normalization
x = Dropout(0.2)(x)
predictions = Dense(7, activation='softmax')(x)  # Assuming 7 emotion classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers initially for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Optionally, unfreeze the last few layers for fine-tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Compile the model with an adaptive optimizer and additional metrics
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Set an initial learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(), Recall()]
)


# Custom data generator with augmentations
def augmented_data_generator(generator, datagen, batch_size=32):
    while True:
        batch_features, batch_labels = next(generator)
        augmented_data = datagen.flow(batch_features, batch_labels, batch_size=batch_size, shuffle=False)
        yield next(augmented_data)

# Training and testing data generators with augmentation applied
train_generator = augmented_data_generator(train_generator, data_gen, 64)
val_generator = augmented_data_generator(test_generator, data_gen, 64)

# Early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=12271 // 64,
    epochs=30,
    callbacks=[early_stopping, reduce_lr],
    validation_data=val_generator,
    validation_steps=12271 // 64,
)


24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/30
191/191 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.1896 - loss: 3.4481 - precision: 0.2013 - recall: 0.0572 - val_accuracy: 0.3861 - val_loss: 2.6708 - val_precision: 0.3861 - val_recall: 0.3861 - learning_rate: 0.0010
Epoch 2/30
191/191 ━━━━━━━━━━━━━━━━━━━━ 286s 2s/step - accuracy: 0.3076 - loss: 2.7417 - precision: 0.3411 - recall: 0.0675 - val_accuracy: 0.3858 - val_loss: 2.4165 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/30
191/191 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.3350 - loss: 2.5127 - precision: 0.3516 - recall: 0.0632 - val_accuracy: 0.3876 - val_loss: 2.3010 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 4/30
191/191 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.3422 - loss: 2.3468 - precision: 0.3175 - recall: 0.0403 - val_accuracy: 0.3864 - val_loss: 2.1775 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learnin

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import label_binarize
from itertools import cycle
import seaborn as sns

def evaluate_classification_model(model, generator, class_names):
    """
    Evaluates a classification model and generates key metrics and visualizations.

    Parameters:
    - model: Trained Keras/TensorFlow model.
    - generator: ImageDataGenerator that yields test data and labels.
    - class_names: List of class names corresponding to emotions.

    Returns:
    - None (generates visualizations and prints metrics).
    """
    # Get predictions and true labels from the generator
    predictions = model.predict(generator)
    predicted_classes = np.argmax(predictions, axis=1)

    # Retrieve true labels from the generator
    true_classes = generator.classes
    true_classes_binarized = label_binarize(true_classes, classes=range(len(class_names)))

    # Generate the classification report
    report = classification_report(true_classes, predicted_classes, target_names=class_names)
    print("Classification Report:\n", report)

    # Confusion Matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    # ROC Curve and AUC
    plt.figure(figsize=(10, 8))
    fpr, tpr, roc_auc = {}, {}, {}
    for i in range(len(class_names)):
        fpr[i], tpr[i], _ = roc_curve(true_classes_binarized[:, i], predictions[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], label=f'ROC curve of class {class_names[i]} (AUC = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Each Emotion')
    plt.legend(loc="lower right")
    plt.show()

    # Precision-Recall Curve
    plt.figure(figsize=(10, 8))
    for i in range(len(class_names)):
        precision, recall, _ = precision_recall_curve(true_classes_binarized[:, i], predictions[:, i])
        plt.plot(recall, precision, label=f'Precision-Recall curve of class {class_names[i]}')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve for Each Emotion')
    plt.legend(loc="lower left")
    plt.show()

    # Training/Validation Curves
    history = model.history.history  # Use the saved `history` attribute from training
    plt.figure(figsize=(14, 5))

    # Accuracy Curve
    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')

    # Loss Curve
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

# Running evaluation with your model, test generator, and emotion labels
evaluate_classification_model(model, test_generator, list(emotion_labels.values()))
